In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Luke\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Reading the News API key enviroment variable

api_key = '755bee183ad0447587d1d607d884ca99'

In [5]:
# Create a newsapi client

newsapi = NewsApiClient(api_key=api_key)

In [26]:
# Fetch all the news about Desktop Metals
dm = newsapi.get_everything(
    q="Desktop Metals",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles Desktop Metals: {dm['totalResults']}")

# Show sample article
dm["articles"][0]

Total articles Desktop Metals: 87


{'source': {'id': None, 'name': 'Motley Fool'},
 'author': 'newsfeedback@fool.com (Rich Smith)',
 'title': 'Why Desktop Metal Popped -- and ExOne Stock Soared',
 'description': "It's earnings day for both companies, and merger day, too.",
 'url': 'https://www.fool.com/investing/2021/08/12/why-desktop-metal-popped-and-exone-stock-soared/',
 'urlToImage': 'https://g.foolcdn.com/editorial/images/639006/wall-painting-depicts-a-large-yellow-fish-eating-a-smaller-yellow-fish.jpg',
 'publishedAt': '2021-08-12T15:25:50Z',
 'content': 'What happened\r\nIt seems The ExOne Company(NASDAQ:XONE) and Desktop Metal(NYSE:DM) are two great tastes that taste great together. Both makers of industrial 3D printing machines, both companies that r… [+2138 chars]'}

In [27]:
# Create the Desktop Metals sentiment scores DataFrame
dm_sentiments = []

for article in dm["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        dm_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
dm_df = pd.DataFrame(dm_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
dm_df = dm_df[cols]

dm_df.head()

,date,text,compound,positive,negative,neutral
0,2021-08-12,What happened\r\nIt seems The ExOne Company(NA...,0.8481,0.220,0.000,0.780
1,2021-08-12,Desktop Metal(NYSE:DM) reported second-quarter...,0.0000,0.000,0.000,1.000
2,2021-08-11,Image source: The Motley Fool.\r\nSierra Metal...,-0.4404,0.000,0.108,0.892
3,2021-08-10,Desktop Metal(NYSE:DM) is slated to report its...,0.0000,0.000,0.000,1.000
4,2021-07-28,3D printing is an additive process in which th...,0.5994,0.133,0.000,0.867


In [28]:
# Get descriptive stats from the DataFrame
dm_df.describe()

,compound,positive,negative,neutral
count,87.000000,87.000000,87.000000,87.000000
mean,0.203989,0.078897,0.029241,0.891839
std,0.389707,0.075936,0.046946,0.088806
min,-0.757900,0.000000,0.000000,0.654000
25%,0.000000,0.000000,0.000000,0.836000
50%,0.077200,0.061000,0.000000,0.900000
75%,0.558950,0.138000,0.054000,1.000000
max,0.891000,0.263000,0.200000,1.000000
